# Dynamics 365 Investigations Notebook

This notebook is designed to help SOC analysts investigate Dynamics 365 activities. Many events generated by the Dynamics 365 connector do not feature friendly names, making it difficult to determine exactly what records were modified and by who. This notebook uses the Dynamics 365 Web API to resolve guids to their friendly name and thus aid with the investigation of alerts or creation of new alert rules.

**Please read the setup pre-requisites at the end of this Notebook before getting started.** 







## Install required packages

In [ ]:
%pip install requests
%pip install msal
%pip install pandas
import ipywidgets as widgets

## Initialize environment variables

Configure the variables in the code block below to match your environment and then execute the cell.

* **tenant**: directory (tenant) ID of your Azure AD.
* **appid**: Azure AD application registration with D365 impersonation permission grant.
* **d365env**: Dynamics 365 Instance URL 

In [ ]:
tenant = "e.g. f9d64s86-6b2d-4297-9625-1f221ba2ef49"
appid = "e.g. 88bd628b-2064-40f0-a065-967586df431e"
d365env = "e.g. https://org50e62e90.crm11.dynamics.com"
d365scope = d365env + "/.default"

print("The following settings will be used by this Notebook.")
print("Azure AD Tenant: " + tenant)
print("Client ID: " + appid)
print("Dynamics 365 Environment: " + d365env)


## Acquire a token from Azure AD

In this step you'll request a token from Azure AD using a device code which will be entered into your web browser. Execute the cell below, copy and paste the code into the link displayed in the cell output. Upon success you should see your username displayed in the output.

In [4]:
import json
import requests
import msal
import pandas
app = msal.PublicClientApplication(
    appid, 
    authority="https://login.microsoftonline.com/" + tenant,
    )
result = None
accounts = app.get_accounts()
if accounts:
    print("Pick the account you want to use to proceed:")
    for a in accounts:
        print(a["username"])
    chosen = accounts[0]
    result = app.acquire_token_silent([d365scope], account=chosen)
if not result:
    flow = app.initiate_device_flow(scopes=[d365scope])
    if "user_code" not in flow:
        raise ValueError(
            "Fail to create device flow. Err: %s" % json.dumps(flow, indent=4))
    print(flow["message"])
    result = app.acquire_token_by_device_flow(flow)    
    print("Token acquired for " + (result['id_token_claims'])['preferred_username'])  

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DXYPE2YJ8 to authenticate.
Token acquired for labadmin@azwk.onmicrosoft.com


## Query Contacts from Dynamics 365 environment

In this example, the fullname and contactid are returned for 10 contacts. You may adjust the query to suit your requirements by changing the **d365query** variable in the code box below. Further information can be found here https://docs.microsoft.com/en-us/dynamics365/customerengagement/on-premises/developer/webapi/samples?view=op-9-1

In [ ]:
d365query = "/api/data/v9.2/contacts?$select=fullname,contactid&$top=5"

import json
import pandas
if "access_token" in result:
    d365_data = requests.get(
        d365env + d365query,
        headers={'Authorization': 'Bearer ' + result['access_token']},).text
    print(pandas.io.json.json_normalize((json.loads(d365_data))['value']))
else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))

## Query Accounts from Dynamics 365 environment

This query example displays all available properties of an Account entity which could be queried using the Dataverse Web API.

In [ ]:
d365query = "/api/data/v9.2/accounts?$top=1"

import json
import pandas
if "access_token" in result:
    d365_data = requests.get(
        d365env + d365query,
        headers={'Authorization': 'Bearer ' + result['access_token']},).text
else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))
json.loads(d365_data)

## List available API entity names

With an extensive list of different entities, both built in and customer defined, understanding the nature of the dataverse in your environment is necessary for successful threat hunting. This query shows all the different entitity types which can be accessed.

In [ ]:
d365query = "/api/data/v9.2/?$select=name"
limitoutput = 30

import json
import pandas
d365_data = None

if "access_token" in result:
    d365_data = requests.get(
        d365env + d365query,
        headers={'Authorization': 'Bearer ' + result['access_token']},).json()
else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))
pandas.set_option('display.max_rows', limitoutput)
pandas.json_normalize(d365_data, record_path=['value'])

## Security configuration state information

Dynamics 365 has a complex Role Based Access Control (RBAC) model which combines not only users and roles, but also the concepts of Business Units and Teams. It is the combination of the collective membership of these entitlements which grant the user access to business data. Use the cell below to select an RBAC entity type to query and run the cell following it to see the results.

In [61]:
entity = widgets.Dropdown(
    options=['businessunits', 'teams', 'roles', 'systemusers'],
    value='businessunits',
    description='Query Type:',
    disabled=False,
)
display(entity)

Dropdown(description='Query Type:', options=('businessunits', 'teams', 'roles', 'systemusers'), value='busines…

In [ ]:
d365query = '/api/data/v9.2/' + entity.value
import json
import pandas
d365_data = None
if "access_token" in result:
    d365_data = requests.get(
        d365env + d365query,
        headers={'Authorization': 'Bearer ' + result['access_token']},).json()  
else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))
pandas.set_option('display.max_rows', None)
pandas.json_normalize(d365_data, record_path=['value'])


# <a id="setup"> Configure Notebook Pre-requisites </a>


This notebook uses an Azure AD app registration with user impersonation permisssions to access the Dynamics Web API on behalf of the user running the notebook. Based on the users level of access within Dynamics, the user will see whatever records their privileges grant them access to. Follow the below steps to create and configure the Azure AD App Registration.

1. Within the Azure AD Portal (https://aad.portal.azure.com), under App Registrations, create a new App Registration. Take note of the **Application (client) ID** and the **Directory (tenant) ID** as these will be used as configuration variables to launch the Notebook.
    <br><br>
    <img src="https://github.com/kingwil/d365Solution/raw/main/images/notebook-1.png" height="300px" alt="Screenshot of Azure AD App Reg">
    <br><br><br>
2. Under Authentication ensure your Web Application has a **redirect URI pointing to your Dynamics 365 instance URL**. This can be obtained from the PowerPlatform Admin center.
    <br><br>
    <img src="https://github.com/kingwil/d365Solution/raw/main/images/notebook-2.png" height="300px" alt="Screenshot of App Reg Auth settings">
    <br><br><br>
3. Also within Authentication, ensure **Public Client** is selected
    <br><br>
    <img src="https://github.com/kingwil/d365Solution/raw/main/images/notebook-3.png" height="300px" alt="Screenshot of App Reg Auth settings">
    <br><br><br>
3. Ensure the **Dynamics 365 User Impersonation permission** is granted.
   <br><br>
   <img src="https://github.com/kingwil/d365Solution/raw/main/images/notebook-4.png" height="300px" alt="Screenshot of App Reg Permissions">

